# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,byron bay,-28.6500,153.6167,27.75,61,92,3.98,AU,1675819711
1,1,rikitea,-23.1203,-134.9692,26.67,68,7,5.00,PF,1675819593
2,2,cockburn town,21.4612,-71.1419,25.48,75,0,10.11,TC,1675819711
3,3,tuatapere,-46.1333,167.6833,12.51,66,81,6.51,NZ,1675819711
4,4,castro,-24.7911,-50.0119,16.99,92,26,2.48,BR,1675819689


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="EsriImagery",size= "Humidity",scale= .45, color= "City", alpha=.75)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
# max temperature lower than 25 degrees but higher than 18
# humidity less than 60%
# Wind speed less than 5 m/s
# cloudiness less than 5

criteria_df = city_data_df.copy()
criteria_df = criteria_df[(criteria_df['Max Temp']<25) & (criteria_df['Max Temp']>18) & \
                          (criteria_df['Humidity']<60) & (criteria_df['Wind Speed']<5) & \
                         (criteria_df['Cloudiness']<5)]

# checking count of values
# criteria_df.count()

# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,busselton,-33.6500,115.3333,22.42,57,0,3.71,AU,1675819712
80,80,rosarito,32.3333,-117.0333,18.01,47,0,2.52,MX,1675819738
152,152,salalah,17.0151,54.0924,20.05,52,0,1.54,OM,1675819600
181,181,carlsbad,33.1581,-117.3506,18.95,46,0,3.60,US,1675819616
350,350,comodoro rivadavia,-45.8667,-67.5000,20.92,32,0,1.54,AR,1675819748
358,358,guerrero negro,27.9769,-114.0611,18.27,30,0,2.89,MX,1675819836
412,412,malakal,9.5334,31.6605,23.54,16,0,4.48,SS,1675819856
414,414,marsabit,2.3284,37.9899,19.71,43,2,2.25,KE,1675819857
463,463,kaabong,3.4836,34.1492,19.64,41,0,2.13,UG,1675819920
480,480,puerto madryn,-42.7692,-65.0385,19.83,51,0,3.74,AR,1675819927


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,busselton,AU,-33.6500,115.3333,57,
80,rosarito,MX,32.3333,-117.0333,47,
152,salalah,OM,17.0151,54.0924,52,
181,carlsbad,US,33.1581,-117.3506,46,
350,comodoro rivadavia,AR,-45.8667,-67.5000,32,
358,guerrero negro,MX,27.9769,-114.0611,30,
412,malakal,SS,9.5334,31.6605,16,
414,marsabit,KE,2.3284,37.9899,43,
463,kaabong,UG,3.4836,34.1492,41,
480,puerto madryn,AR,-42.7692,-65.0385,51,


In [26]:
#### TESTING API ####
# latitude = -33.6500
# longitude = 115.3333
# radius = 10000
# categories = "accommodation.hotel"
# limit = 1
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"
# params = {
#     "categories":categories,
#     "limit":limit,
#     "filter":filters,
#     "bias":bias,
#     "apiKey":geoapify_key
# }

# base_url = "https://api.geoapify.com/v2/places"
# response = requests.get(base_url,params=params).json()
# pprint(response)

{'features': [{'geometry': {'coordinates': [115.27977417993365,
                                            -33.659569140351024],
                            'type': 'Point'},
               'properties': {'address_line1': 'Broadwater Beach Resort',
                              'address_line2': 'Holgate Road, Broadwater WA, '
                                               'Australia',
                              'categories': ['accommodation',
                                             'accommodation.hotel',
                                             'building',
                                             'building.accommodation',
                                             'internet_access'],
                              'city': 'City Of Busselton',
                              'country': 'Australia',
                              'country_code': 'au',
                              'datasource': {'attribution': '© OpenStreetMap '
                                            

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
busselton - nearest hotel: Broadwater Beach Resort
rosarito - nearest hotel: WorldMark La Paloma
salalah - nearest hotel: Muscat International Hotel
carlsbad - nearest hotel: Carlsbad Inn Beach Resort
comodoro rivadavia - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
malakal - nearest hotel: No hotel found
marsabit - nearest hotel: Chicho Hotel
kaabong - nearest hotel: Sacavian
puerto madryn - nearest hotel: Hotel Gran Palace


,City,Country,Lat,Lng,Humidity,Hotel Name
5,busselton,AU,-33.6500,115.3333,57,Broadwater Beach Resort
80,rosarito,MX,32.3333,-117.0333,47,WorldMark La Paloma
152,salalah,OM,17.0151,54.0924,52,Muscat International Hotel
181,carlsbad,US,33.1581,-117.3506,46,Carlsbad Inn Beach Resort
350,comodoro rivadavia,AR,-45.8667,-67.5000,32,No hotel found
358,guerrero negro,MX,27.9769,-114.0611,30,Plaza sal paraiso
412,malakal,SS,9.5334,31.6605,16,No hotel found
414,marsabit,KE,2.3284,37.9899,43,Chicho Hotel
463,kaabong,UG,3.4836,34.1492,41,Sacavian
480,puerto madryn,AR,-42.7692,-65.0385,51,Hotel Gran Palace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",size= "Humidity",scale= 1.5, \
                                   color= "City", alpha=.75,hover_cols=["Hotel Name","Country"])

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)